In [1]:
import numpy as np
import plotly.graph_objects as go

The inputs to the HyperNet will be Amplitude and Frequency. These variables define the range and number of samples for the Amplitude and Frequency arrays.

In [2]:
NUM_AMP_PTS = 15
MIN_AMP = 0.1
MAX_AMP = 0.9

NUM_FREQ_PTS = 15
MIN_FREQ = 2
MAX_FREQ = 10

In [3]:
SEQ_LEN = 200
PRED_LEN = 1
TRAIN_STEP = 5
TEST_STEP = 1
ARRAY_LEN = 1000
ARRAYS_PER_PARAM_SET = 15

In [4]:
NOISE_FACTOR = 0.05

In [5]:
NTEP = 5
NTNP = 5
NVEP = 5
NVNP = 5

In [6]:
TRAIN_FILE = "sinusoidal-train.npz"
TEST1_FILE = "sinusoidal-test-tp.npz"
TEST2_FILE = "sinusoidal-test-np.npz"
VAL_1_FILE = "sinusoidal-val-tp.npz"
VAL_2_FILE = "sinusoidal-val-np.npz"

In [7]:
A = np.linspace(MIN_AMP, MAX_AMP, NUM_AMP_PTS)
F = np.linspace(MIN_FREQ, MAX_FREQ, NUM_FREQ_PTS)
t = np.linspace(0, 2 * np.pi, ARRAY_LEN)

In [8]:
def sequence_sine(sine_: np.ndarray, step: int) -> tuple[np.ndarray]:
    seqs, preds, start = None, None, 0
    for _ in range(int((ARRAY_LEN - SEQ_LEN - PRED_LEN) / step) + 1):
        seq = sine_[start : start + SEQ_LEN].reshape(1, SEQ_LEN, 1)
        pred = sine_[start + SEQ_LEN : start + SEQ_LEN + PRED_LEN].reshape(
            1,
        )
        seqs = seq if seqs is None else np.vstack([seqs, seq])
        preds = pred if preds is None else np.vstack([preds, pred])
        start += step
    return seqs, preds

In [13]:
train_ds, count = [], 0
for amp in A:
    for f in F:
        hyper_params = np.array([amp, f])
        for i in range(ARRAYS_PER_PARAM_SET):
            sine = amp * np.sin(f * t) + (amp * NOISE_FACTOR * (np.random.rand(*t.shape) - 0.5))
            seqs, preds = sequence_sine(sine, TRAIN_STEP)
            train_ds.append({"hx": hyper_params, "tx": seqs, "tyhat": preds})
            count += 1
            print(f"Created and sequenced {count} arrays")

print(f"Writing training data set to {TRAIN_FILE}")
np.savez_compressed(TRAIN_FILE, train_ds)


# inspect the last sine wave
fig = go.Figure()
fig.update_layout(
    xaxis={
        "tickmode": "array",
        "tickvals": [0.5 * np.pi, np.pi, 1.5 * np.pi, 2 * np.pi],
        "ticktext": ["π/2", "π", "3π/2", "2π"],
    },
)
fig.add_trace(
    go.Scatter(
        x=t,
        y=sine,
    )
)
fig.show()

Created and sequenced 1 arrays
Created and sequenced 2 arrays
Created and sequenced 3 arrays
Created and sequenced 4 arrays
Created and sequenced 5 arrays
Created and sequenced 6 arrays
Created and sequenced 7 arrays
Created and sequenced 8 arrays
Created and sequenced 9 arrays
Created and sequenced 10 arrays
Created and sequenced 11 arrays
Created and sequenced 12 arrays
Created and sequenced 13 arrays
Created and sequenced 14 arrays
Created and sequenced 15 arrays
Created and sequenced 16 arrays
Created and sequenced 17 arrays
Created and sequenced 18 arrays
Created and sequenced 19 arrays
Created and sequenced 20 arrays
Created and sequenced 21 arrays
Created and sequenced 22 arrays
Created and sequenced 23 arrays
Created and sequenced 24 arrays
Created and sequenced 25 arrays
Created and sequenced 26 arrays
Created and sequenced 27 arrays
Created and sequenced 28 arrays
Created and sequenced 29 arrays
Created and sequenced 30 arrays
Created and sequenced 31 arrays
Created and seque

In [10]:
A_sels, F_sels, test_1, val_1 = [], [], [], []

for _ in range(NTEP):

    # select random A and F from HyperNet training parameters
    sel_A = np.random.randint(0, len(A) - 1, (100,))
    sel_F = np.random.randint(0, len(F) - 1, (100,))
    sel_A = [x for x in sel_A if x not in A_sels][0]
    sel_F = [x for x in sel_F if x not in F_sels][0]
    A_sels.append(sel_A)
    F_sels.append(sel_F)

    # create tensors
    hyper_params = np.array([A[sel_A], F[sel_F]])
    sine = A[sel_A] * np.sin(F[sel_F] * t)

    # sequence sine wave array
    seqs, preds = sequence_sine(sine, TRAIN_STEP)

    # append data set
    test_1.append({"hx": hyper_params, "tx": seqs, "tyhat": preds})

for _ in range(NVEP):

    # select random A and F from training HyperNet Parameters
    sel_A = np.random.randint(0, len(A) - 1, (100,))
    sel_F = np.random.randint(0, len(F) - 1, (100,))
    sel_A = [x for x in sel_A if x not in A_sels][0]
    sel_F = [x for x in sel_F if x not in F_sels][0]
    A_sels.append(sel_A)
    F_sels.append(sel_F)

    # create tensors
    hyper_params = np.array([A[sel_A], F[sel_F]])
    sine = A[sel_A] * np.sin(F[sel_F] * t)

    # sequence sine wave array
    seqs, preds = sequence_sine(sine, TEST_STEP)

    # append data set
    val_1.append({"hx": hyper_params, "tx": seqs, "tyhat": preds})

np.savez_compressed(TEST1_FILE, test_1)
np.savez_compressed(VAL_1_FILE, val_1)

In [11]:
min_A, max_A = min(A), max(A)
min_F, max_F = min(F), max(F)

A_sels, F_sels, test_2, val_2 = [], [], [], []

for _ in range(NTNP):

    # select random A and F not in HyperNet training parameters
    sel_A = np.random.rand(1000)
    sel_F = np.random.rand(1000) * max_F
    sel_A = [x for x in sel_A if x not in A and x not in A_sels and min_A < x < max_A][0]
    sel_F = [x for x in sel_F if x not in F and x not in F_sels and min_F < x < max_F][0]
    A_sels.append(sel_A)
    F_sels.append(sel_F)

    # create tensors
    hyper_params = np.array([sel_A, sel_F])
    sine = sel_A * np.sin(sel_F * t)

    # sequence sine wave array
    seqs, preds = sequence_sine(sine, TRAIN_STEP)

    # append data set
    test_2.append({"hx": hyper_params, "tx": seqs, "tyhat": preds})

for _ in range(NVNP):

    # select random A and F not in HyperNet training parameters
    sel_A = np.random.rand(1000)
    sel_F = np.random.rand(1000) * max_F
    sel_A = [x for x in sel_A if x not in A and x not in A_sels and min_A < x < max_A][0]
    sel_F = [x for x in sel_F if x not in F and x not in F_sels and min_F < x < max_F][0]
    A_sels.append(sel_A)
    F_sels.append(sel_F)

    # create tensors
    hyper_params = np.array([sel_A, sel_F])
    sine = sel_A * np.sin(sel_F * t)

    # sequence sine wave array
    seqs, preds = sequence_sine(sine, TEST_STEP)

    # append data set
    val_2.append({"hx": hyper_params, "tx": seqs, "tyhat": preds})

np.savez_compressed(TEST2_FILE, test_2)
np.savez_compressed(VAL_2_FILE, val_2)